# Modulo 06 - Python Avançado


In [42]:
import pandas as pd
import numpy as np
import re

In [43]:
data = pd.read_csv('C:/Users/pedro/OneDrive/Documentos/CURSOS/Python do DS ao DEV/products_hm.csv')

In [44]:
data.head()

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id,color_name,fit,size,product safety,composition
0,636207006.0,men_jeans_slim,Slim Jeans,$ 19.99,2021-04-11 17:48:05,636207.0,6.0,Dark denim blue,Slim fit,NaN,NaN,"Cotton 98%, Elastane 2%"
1,636207006.0,men_jeans_slim,Slim Jeans,$ 19.99,2021-04-11 17:48:05,636207.0,6.0,Dark gray denim,Slim fit,NaN,NaN,"Cotton 98%, Elastane 2%"
2,636207006.0,men_jeans_slim,Slim Jeans,$ 19.99,2021-04-11 17:48:05,636207.0,6.0,Denim blue,Slim fit,NaN,NaN,"Cotton 98%, Elastane 2%"
3,636207006.0,men_jeans_slim,Slim Jeans,$ 19.99,2021-04-11 17:48:05,636207.0,6.0,Gray,Slim fit,NaN,NaN,"Cotton 98%, Elastane 2%"
4,636207006.0,men_jeans_slim,Slim Jeans,$ 19.99,2021-04-11 17:48:05,636207.0,6.0,Light denim blue,Slim fit,NaN,NaN,"Cotton 98%, Elastane 2%"


In [45]:
# product id
data = data.dropna(subset=['product_id'])
data['product_id'] = data['product_id'].astype(int)

# product name
data['product_name'] = data['product_name'].apply(lambda x: x.replace(' ', '_').lower())

# product price
data['product_price'] = data['product_price'].apply(lambda x: x.replace('$', '')).astype(float)

# scrapy datetime
data['scrapy_datetime'] = pd.to_datetime(data['scrapy_datetime'], format='%Y-%m-%d %H:%M:%S')

# style id
data['style_id'] = data['style_id'].astype(int)

# color id
data['color_id'] = data['color_id'].astype(int)

# color name
data['color_name'] = data['color_name'].apply(lambda x: x.replace(' ', '_').replace('/', '_').lower() if pd.notnull(x) else x)

# fit
data['fit'] = data['fit'].apply(lambda x: x.replace(' ', '_').lower() if pd.notnull(x) else x)

# size number
data['size_number'] = data['size'].apply(lambda x: re.search('\d{3}cm', x).group(0) if pd.notnull(x) else x)
data['size_number'] = data['size_number'].apply(lambda x: re.search('\d+', x).group(0) if pd.notnull(x) else x)

# size model
data['size_model'] = data['size'].str.extract('(\d+/\\d+)'
                                             )
data = data.drop(columns=['size', 'product safety'], axis=1)

# composition
data = data[~data['composition'].str.contains('Pocket lining:', na=False)]
data = data[~data['composition'].str.contains('Lining:', na=False)]
data = data[~data['composition'].str.contains('Shell:', na=False)]

# break composition by comma
df1 = data['composition'].str.split(',', expand=True)

In [46]:
# cotton | polyester | elastano | elasterell
df_ref = pd.DataFrame(index=np.arange(len(data)), columns=['cotton', 'polyester', 'elastane', 'elasterell'])

# cotton
df_cotton = df1[0]
df_cotton.name = 'cotton'
df_ref = pd.concat([df_ref, df1[0]], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]

# polyester
df_polyester = df1.loc[df1[1].str.contains('Polyester', na=True), 1]
df_polyester.name = 'polyester'
df_ref = pd.concat([df_ref, df_polyester], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]

# elastano
df_elastane = df1.loc[df1[1].str.contains('Elastane', na=True), 1]
df_elastane.name = 'elastane'
df_ref = pd.concat([df_ref, df_elastane], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]

# elasterell
df_elasterell = df1.loc[df1[1].str.contains('Elasterell', na=True), 1]
df_elasterell.name = 'elasterell'
df_ref = pd.concat([df_ref, df_elasterell], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]

# final join
data = pd.concat([data, df_ref], axis=1)

# format composition data
data['cotton'] = data['cotton'].apply(lambda x: int(re.search('\d+', x).group(0)) / 100 if pd.notnull(x) else x)
data['polyester'] = data['polyester'].apply(lambda x: int(re.search('\d+', x).group(0)) / 100 if pd.notnull(x) else x)
data['elastane'] = data['elastane'].apply(lambda x: int(re.search('\d+', x).group(0)) / 100 if pd.notnull(x) else x)
data['elasterell'] = data['elasterell'].apply(lambda x: int(re.search('\d+', x).group(0)) / 100 if pd.notnull(x) else x)

data.head()

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id,color_name,fit,composition,size_number,size_model,cotton,polyester,elastane,elasterell
0,636207006.0,men_jeans_slim,slim_jeans,19.99,2021-04-11 17:48:05,636207.0,6.0,dark_denim_blue,slim_fit,"Cotton 98%, Elastane 2%",NaN,NaN,0.98,NaN,0.02,NaN
1,636207006.0,men_jeans_slim,slim_jeans,19.99,2021-04-11 17:48:05,636207.0,6.0,dark_gray_denim,slim_fit,"Cotton 98%, Elastane 2%",NaN,NaN,0.98,NaN,0.02,NaN
2,636207006.0,men_jeans_slim,slim_jeans,19.99,2021-04-11 17:48:05,636207.0,6.0,denim_blue,slim_fit,"Cotton 98%, Elastane 2%",NaN,NaN,0.98,NaN,0.02,NaN
3,636207006.0,men_jeans_slim,slim_jeans,19.99,2021-04-11 17:48:05,636207.0,6.0,gray,slim_fit,"Cotton 98%, Elastane 2%",NaN,NaN,0.98,NaN,0.02,NaN
4,636207006.0,men_jeans_slim,slim_jeans,19.99,2021-04-11 17:48:05,636207.0,6.0,light_denim_blue,slim_fit,"Cotton 98%, Elastane 2%",NaN,NaN,0.98,NaN,0.02,NaN
